In [2]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
!hostname
!pwd

dv002.bridges2.psc.edu
/ocean/projects/asc170022p/mtragoza/mre-pinn/MICCAI-2023


In [3]:
import sys, os, tqdm
import numpy as np
import xarray as xr
import torch

os.environ['DDEBACKEND'] = 'pytorch'
import deepxde

sys.path.append('..')
%aimport mre_pinn

sys.path.append('../../mre_ai')
#import mre_ai.mre_datasets

torch.cuda.is_available()

True

## Process images for a single patient

- load SITK images from NIFTI files
- perform filtering/inpainting on wave image
- register images wrt anatomical image
- load segmentation model
- perform image segmentation
- convert images to xarray Dataset

In [4]:
%autoreload
patient = mre_pinn.data.ImagingPatient('0006')
patient.load_images()
patient.metadata

Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/t1_pre_in.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/t1_pre_water.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/t1_pre_out.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/t1_pre_fat.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/t2.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/mre_raw.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/wave.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/mre.nii


size    spacing      origin       limit      center  \
sequence     dimension                                                        
t1_pre_in    0           512   0.703100 -204.130005  155.154108  -24.487948   
             1           512   0.703100 -208.591995  150.692118  -28.949939   
             2            88   2.300003 -142.013000   58.087265  -41.962868   
t1_pre_water 0           512   0.703100 -204.130005  155.154108  -24.487948   
             1           512   0.703100 -208.591995  150.692118  -28.949939   
             2            88   2.300003 -142.013000   58.087265  -41.962868   
t1_pre_out   0           512   0.703100 -204.130005  155.154108  -24.487948   
             1           512   0.703100 -208.591995  150.692118  -28.949939   
             2            88   2.300003 -142.013000   58.087265  -41.962868   
t1_pre_fat   0           512   0.703100 -204.130005  155.154108  -24.487948   
             1           512   0.703100 -208.591995  150.692118  -28.949939   
             2            88   2.300003 -142.013000   58.087265  -41.962868   
t2           0           256   1.406300 -200.263000  158.343486  -20.959757   
             1           256   1.406300 -208.240997  150.365490  -28.937754   
             2            23  10.000000 -154.516006   65.483994  -44.516006   
mre_raw      0           256   1.406300 -206.904007  151.702480  -27.600763   
             1           256   1.406300 -195.858994  162.747493  -16.555750   
             2             4   1.000000    0.000000    3.000000    1.500000   
wave         0           512   1.000000    0.000000  511.000000  255.500000   
             1           512   1.000000    0.000000  511.000000  255.500000   
             2             4   1.000000    0.000000    3.000000    1.500000   
mre          0           256   1.406300 -206.904007  151.702480  -27.600763   
             1           256   1.406300 -195.858994  162.747493  -16.555750   
             2             4  11.000000   -1.056400   31.943600   15.443600   

                            extent  
sequence     dimension              
t1_pre_in    0          359.987213  
             1          359.987213  
             2          202.400269  
t1_pre_water 0          359.987213  
             1          359.987213  
             2          202.400269  
t1_pre_out   0          359.987213  
             1          359.987213  
             2          202.400269  
t1_pre_fat   0          359.987213  
             1          359.987213  
             2          202.400269  
t2           0          360.012787  
             1          360.012787  
             2          230.000000  
mre_raw      0          360.012787  
             1          360.012787  
             2            4.000000  
wave         0          512.000000  
             1          512.000000  
             2            4.000000  
mre          0          360.012787  
             1          360.012787  
             2           44.000000

In [6]:
%autoreload
patient.preprocess(same_grid=True, mre_size=(256, 256, 4), anat_size=(256, 256, 4))

Correcting metadata on mre_raw
Correcting metadata on wave
Restoring wave
Registering t1_pre_out to mre_raw
ParameterObject (0x55905def4f30)
  RTTI typeinfo:   elastix::ParameterObject
  Reference Count: 1
  Modified Time: 36498
  Debug: Off
  Object Name: 
  Observers: 
    none
ParameterMap 0: 
  (CenterOfRotationPoint -27.6008 -16.5558 15.4436)
  (CompressResultImage "false")
  (ComputeZYX "false")
  (DefaultPixelValue 0)
  (Direction 1 0 0 0 1 0 0 0 1)
  (FinalBSplineInterpolationOrder 3)
  (FixedImageDimension 3)
  (FixedInternalImagePixelType "float")
  (HowToCombineTransforms "Compose")
  (Index 0 0 0)
  (InitialTransformParametersFileName "NoInitialTransform")
  (MovingImageDimension 3)
  (MovingInternalImagePixelType "float")
  (NumberOfParameters 6)
  (Origin -207.243 -196.198 -84.6065)
  (ResampleInterpolator "FinalBSplineInterpolator")
  (Resampler "DefaultResampler")
  (ResultImageFormat "nii")
  (ResultImagePixelType "float")
  (Size 512 512 88)
  (Spacing 0.7031 0.7031 2

  (UseDirectionCosines "true")
Resizing t1_pre_in to (256, 256, 4)
Resizing t1_pre_water to (256, 256, 4)
Resizing t1_pre_out to (256, 256, 4)
Resizing t1_pre_fat to (256, 256, 4)
Resizing t2 to (256, 256, 4)
Resizing mask to (256, 256, 4)
Resizing mre_raw to (256, 256, 4)
Resizing wave to (256, 256, 4)
Resizing mre to (256, 256, 4)
Resizing mask to (256, 256, 4)
Correcting metadata on t1_pre_in
Correcting metadata on t1_pre_water
Correcting metadata on t1_pre_out
Correcting metadata on t1_pre_fat
Correcting metadata on t2
Correcting metadata on mask
Correcting metadata on mre_raw
Correcting metadata on wave
Correcting metadata on mre
Correcting metadata on mask


In [7]:
patient.metadata

size  spacing      origin       limit     center  \
sequence     dimension                                                     
t1_pre_in    0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   
t1_pre_water 0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   
t1_pre_out   0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   
t1_pre_fat   0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   
t2           0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   
mre_raw      0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   
wave         0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   
mre          0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   
anat_mask    0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   
mre_mask     0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   

                            extent  
sequence     dimension              
t1_pre_in    0          360.012787  
             1          360.012787  
             2           44.000000  
t1_pre_water 0          360.012787  
             1          360.012787  
             2           44.000000  
t1_pre_out   0          360.012787  
             1          360.012787  
             2           44.000000  
t1_pre_fat   0          360.012787  
             1          360.012787  
             2           44.000000  
t2           0          360.012787  
             1          360.012787  
             2           44.000000  
mre_raw      0          360.012787  
             1          360.012787  
             2           44.000000  
wave         0          360.012787  
             1          360.012787  
             2           44.000000  
mre          0          360.012787  
             1          360.012787  
             2           44.000000  
anat_mask    0          360.012787  
             1          360.012787  
             2           44.000000  
mre_mask     0          360.012787  
             1          360.012787  
             2           44.000000

In [15]:
%autoreload
example = mre_pinn.data.MREExample.from_patient(patient)
example.anat

Converting t1_pre_in to xarray
Converting t1_pre_water to xarray
Converting t1_pre_out to xarray
Converting t1_pre_fat to xarray
Converting t2 to xarray
Converting mre_raw to xarray
Converting wave to xarray
Converting mre to xarray
Converting mask to xarray
Converting mask to xarray


<xarray.DataArray 't1_pre_in' (x: 256, y: 256, z: 4, component: 5)>
array([[[[ 3.91698034e-20, -4.19973473e-10,  2.30173620e-20,
          -5.22206455e-20, -7.72612035e-26],
         [-1.41440001e-19, -3.83624965e-10, -3.12337228e-20,
          -4.68830462e-20, -6.98464841e-26],
         [ 5.52396908e-21, -3.28948008e-10, -8.09280424e-20,
          -1.19850354e-19,  9.00861743e-27],
         [-2.39978699e-20, -2.44738230e-10, -3.27558009e-21,
           1.33647933e-19,  7.41968170e-28]],

        [[ 5.19202142e-19,  2.25905669e-10,  3.06531034e-19,
          -7.13152679e-19,  2.95008242e-25],
         [-1.96586421e-18,  2.49976456e-10, -4.59558439e-19,
          -6.74966610e-19,  2.72294940e-25],
         [ 1.11043285e-19,  2.37245112e-10, -1.09133412e-18,
          -1.66069402e-18, -3.54719102e-26],
         [-3.58104220e-19,  1.02129423e-10, -6.64715167e-20,
           1.86872591e-18, -2.92639720e-27]],

        [[ 6.90299716e-18, -9.30554189e-10,  4.08365585e-18,
          -9.73233164e-18, -1.12221213e-24],
...
         [ 0.00000000e+00,  2.88539313e-19,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  7.81875349e-20,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00, -5.60750261e-21,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00, -1.73219731e-20,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00, -7.27389737e-20,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  8.79544700e-21,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00, -7.95772740e-22,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00, -1.77745422e-21,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00, -6.71476423e-21,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00]]]], dtype=float32)
Coordinates:
  * x          (x) float64 -0.4138 -0.411 -0.4082 ... 0.2978 0.3006 0.3034
  * y          (y) float64 -0.3917 -0.3889 -0.3861 ... 0.3199 0.3227 0.3255
  * z          (z) float64 -0.002113 0.01989 0.04189 0.06389
  * component  (component) <U12 't1_pre_in' 't1_pre_out' ... 't1_pre_fat' 't2'
    region     (x, y, z) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [16]:
example.wave

<xarray.DataArray 'wave' (x: 256, y: 256, z: 4)>
array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
...
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 2.59313541e-25,  9.06511984e-21,  3.10481994e-26,
          7.32113561e-27],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]]])
Coordinates:
  * x        (x) float64 -0.4138 -0.411 -0.4082 -0.4054 ... 0.2978 0.3006 0.3034
  * y        (y) float64 -0.3917 -0.3889 -0.3861 ... 0.3199 0.3227 0.3255
  * z        (z) float64 -0.002113 0.01989 0.04189 0.06389
    region   (x, y, z) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [17]:
example.save_xarrays('../data/NAFLD/test/')

Writing ../data/NAFLD/test/0006/wave.nc
Writing ../data/NAFLD/test/0006/mre.nc
Writing ../data/NAFLD/test/0006/mre_mask.nc
Writing ../data/NAFLD/test/0006/anat.nc
Writing ../data/NAFLD/test/0006/anat_mask.nc


In [21]:
%autoreload
example = mre_pinn.data.MREExample.load_xarrays('../data/NAFLD/test/', '0006', anat=True)
example.anat.field.values().shape

Loading ../data/NAFLD/test/0006/wave.nc
Loading ../data/NAFLD/test/0006/mre.nc
Loading ../data/NAFLD/test/0006/mre_mask.nc
Loading ../data/NAFLD/test/0006/anat.nc
Loading ../data/NAFLD/test/0006/anat_mask.nc


(262144, 5)

## Process images for a patient cohort

In [22]:
with open('/ocean/projects/asc170022p/shared/Data/MRE/gold.txt') as f:
    patient_ids = [line.split('.')[0].split('_')[1] for line in f]

len(patient_ids)

181

In [23]:
%autoreload
cohort = mre_pinn.data.ImagingCohort(patient_ids='{' + ','.join(patient_ids) + '}')
len(cohort)

/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0830 is missing sequences {'wave'}
/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0838 is missing sequences {'wave'}
/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0846 is missing sequences {'wave'}
/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0860 is missing sequences {'wave'}
/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0872 is missing sequences {'wave'}
/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1474 is missing sequences {'wave'}
/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1483 is missing sequences {'wave'}
/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1495 is missing sequences {'wave'}
/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1504 is missing sequences {'wave'}
/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1526 

155

In [24]:
%%time
cohort.load_images()

  0%|          | 0/155 [00:00<?, ?it/s]Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0898/t1_pre_in.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0898/t1_pre_water.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0898/t1_pre_out.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0898/t1_pre_fat.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0898/t2.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0898/mre_raw.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0898/wave.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0898/mre.nii
  1%|          | 1/155 [00:02<06:58,  2.72s/it]Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0900/t1_pre_in.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFT

Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0989/t1_pre_fat.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0989/t2.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0989/mre_raw.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0989/wave.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0989/mre.nii
  7%|▋         | 11/155 [00:21<04:07,  1.72s/it]Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0995/t1_pre_in.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0995/t1_pre_water.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0995/t1_pre_out.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0995/t1_pre_fat.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0995/t2.nii
Loading /ocean/projects

Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1103/mre.nii
 14%|█▎        | 21/155 [00:34<02:52,  1.29s/it]Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1106/t1_pre_in.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1106/t1_pre_water.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1106/t1_pre_out.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1106/t1_pre_fat.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1106/t2.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1106/mre_raw.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1106/wave.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1106/mre.nii
 14%|█▍        | 22/155 [00:35<02:54,  1.31s/it]Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/

Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1207/t1_pre_out.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1207/t1_pre_fat.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1207/t2.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1207/mre_raw.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1207/wave.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1207/mre.nii
 21%|██        | 32/155 [00:51<02:53,  1.41s/it]Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1215/t1_pre_in.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1215/t1_pre_water.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1215/t1_pre_out.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/1215/t1_pre_fat.nii
Loading /ocean/

Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-22-19/NIFTI/1337/mre.nii
 27%|██▋       | 42/155 [01:10<03:01,  1.60s/it]Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-22-19/NIFTI/1341/t1_pre_in.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-22-19/NIFTI/1341/t1_pre_water.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-22-19/NIFTI/1341/t1_pre_out.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-22-19/NIFTI/1341/t1_pre_fat.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-22-19/NIFTI/1341/t2.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-22-19/NIFTI/1341/mre_raw.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-22-19/NIFTI/1341/wave.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-22-19/NIFTI/1341/mre.nii
 28%|██▊       | 43/155 [01:15<04:29,  2.40s/it]Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-22-19/

...

Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/0693/t1_pre_out.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/0693/t1_pre_fat.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/0693/t2.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/0693/mre_raw.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/0693/wave.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/0693/mre.nii
 68%|██████▊   | 106/155 [02:49<01:09,  1.43s/it]Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/0704/t1_pre_in.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/0704/t1_pre_water.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/0704/t1_pre_out.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/0704/t1_pre_fat.nii
Loading /ocean/projects/

 75%|███████▍  | 116/155 [03:05<01:02,  1.60s/it]Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1694/t1_pre_in.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1694/t1_pre_water.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1694/t1_pre_out.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1694/t1_pre_fat.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1694/t2.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1694/mre_raw.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1694/wave.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1694/mre.nii
 75%|███████▌  | 117/155 [03:07<01:10,  1.85s/it]Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1699/t1_pre_in.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NI

Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1748/t1_pre_fat.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1748/t2.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1748/mre_raw.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1748/wave.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1748/mre.nii
 82%|████████▏ | 127/155 [03:24<00:53,  1.90s/it]Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1785/t1_pre_in.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1785/t1_pre_water.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1785/t1_pre_out.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1785/t1_pre_fat.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1785/t2.nii
Loading /ocean/projects/asc17002

Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1819/mre.nii
 88%|████████▊ | 137/155 [03:43<00:42,  2.33s/it]Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1829/t1_pre_in.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1829/t1_pre_water.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1829/t1_pre_out.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1829/t1_pre_fat.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1829/t2.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1829/mre_raw.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1829/wave.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1829/mre.nii
 89%|████████▉ | 138/155 [03:44<00:34,  2.02s/it]Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/18

Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1940/t1_pre_fat.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1940/t2.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1940/mre_raw.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1940/wave.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1940/mre.nii
 95%|█████████▌| 148/155 [04:01<00:13,  2.00s/it]Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1948/t1_pre_in.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1948/t1_pre_water.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1948/t1_pre_out.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1948/t1_pre_fat.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_9-12-19/NIFTI/1948/t2.nii
Loading /ocean/projects/asc17002

In [25]:
%%time
cohort.preprocess()

Loading segmentation model
  0%|          | 0/155 [00:00<?, ?it/s]Correcting metadata on mre_raw
Correcting metadata on wave
Restoring wave
Registering t1_pre_out to mre_raw
ParameterObject (0x559136fcea70)
  RTTI typeinfo:   elastix::ParameterObject
  Reference Count: 1
  Modified Time: 581167
  Debug: Off
  Object Name: 
  Observers: 
    none
ParameterMap 0: 
  (CenterOfRotationPoint -3.538 -45.776 158.963)
  (CompressResultImage "false")
  (ComputeZYX "false")
  (DefaultPixelValue 0)
  (Direction 1 0 0 0 1 0 0 0 1)
  (FinalBSplineInterpolationOrder 3)
  (FixedImageDimension 3)
  (FixedInternalImagePixelType "float")
  (HowToCombineTransforms "Compose")
  (Index 0 0 0)
  (InitialTransformParametersFileName "NoInitialTransform")
  (MovingImageDimension 3)
  (MovingInternalImagePixelType "float")
  (NumberOfParameters 6)
  (Origin -213.125 -255.363 22.1128)
  (ResampleInterpolator "FinalBSplineInterpolator")
  (Resampler "DefaultResampler")
  (ResultImageFormat "nii")
  (ResultImagePi

Resizing mre_raw to (256, 256, 4)
Resizing wave to (256, 256, 4)
Resizing mre to (256, 256, 4)
Resizing mask to (256, 256, 4)
Correcting metadata on t1_pre_in
Correcting metadata on t1_pre_water
Correcting metadata on t1_pre_out
Correcting metadata on t1_pre_fat
Correcting metadata on t2
Correcting metadata on mask
Correcting metadata on mre_raw
Correcting metadata on wave
Correcting metadata on mre
Correcting metadata on mask
  1%|          | 1/155 [01:00<2:34:14, 60.10s/it]Correcting metadata on mre_raw
Correcting metadata on wave
Restoring wave
Registering t1_pre_out to mre_raw
ParameterObjectTransforming t1_pre_out
 (0x559136fc4dd0)
  RTTI typeinfo:   elastix::ParameterObject
  Reference Count: 1
  Modified Time: 855560
  Debug: Off
  Object Name: 
  Observers: 
    none
ParameterMap 0: 
  (CenterOfRotationPoint 8.49001 -40.622 97.0866)
  (CompressResultImage "false")
  (ComputeZYX "false")
  (DefaultPixelValue 0)
  (Direction 1 0 0 0 1 0 0 0 1)
  (FinalBSplineInterpolationOrder 3)

ParameterObject (0x5590ea2e61b0)
  RTTI typeinfo:   elastix::ParameterObject
  Reference Count: 1
  Modified Time: 1095179
  Debug: Off
  Object Name: 
  Observers: 
    none
ParameterMap 0: 
  (CenterOfRotationPoint 8.49001 -40.622 97.0866)
  (CompressResultImage "false")
  (ComputeZYX "false")
  (DefaultPixelValue 0)
  (Direction 1 0 0 0 1 0 0 0 1)
  (FinalBSplineInterpolationOrder 3)
  (FixedImageDimension 3)
  (FixedInternalImagePixelType "float")
  (HowToCombineTransforms "Compose")
  (Index 0 0 0)
  (InitialTransformParametersFileName "NoInitialTransform")
  (MovingImageDimension 3)
  (MovingInternalImagePixelType "float")
  (NumberOfParameters 6)
  (Origin -180.771 -229.883 -7.9134)
  (ResampleInterpolator "FinalBSplineInterpolator")
  (Resampler "DefaultResampler")
  (ResultImageFormat "nii")
  (ResultImagePixelType "float")
  (Size 256 256 22)
  (Spacing 1.4844 1.4844 10)
  (Transform "EulerTransform")
  (TransformParameters 0.00300271 -0.000472466 0.00449482 -0.394678 -0.5250

ParameterObject (0x5590e9964990)
  RTTI typeinfo:   elastix::ParameterObject
  Reference Count: 1
  Modified Time: 1334676
  Debug: Off
  Object Name: 
  Observers: 
    none
ParameterMap 0: 
  (CenterOfRotationPoint -14.1432 -20.0002 129.457)
  (CompressResultImage "false")
  (ComputeZYX "false")
  (DefaultPixelValue 0)
  (Direction 1 0 0 0 1 0 0 0 1)
  (FinalBSplineInterpolationOrder 3)
  (FixedImageDimension 3)
  (FixedInternalImagePixelType "float")
  (HowToCombineTransforms "Compose")
  (Index 0 0 0)
  (InitialTransformParametersFileName "NoInitialTransform")
  (MovingImageDimension 3)
  (MovingInternalImagePixelType "float")
  (NumberOfParameters 6)
  (Origin -213.765 -219.622 1.80725)
  (ResampleInterpolator "FinalBSplineInterpolator")
  (Resampler "DefaultResampler")
  (ResultImageFormat "nii")
  (ResultImagePixelType "float")
  (Size 512 512 112)
  (Spacing 0.7813 0.7813 2.3)
  (Transform "EulerTransform")
  (TransformParameters -0.0125404 0.0946621 0.0159711 3.09723 -7.18535 

Registering t1_pre_out to t1_pre_out
ParameterObject (0x559136fb5910)
  RTTI typeinfo:   elastix::ParameterObject
  Reference Count: 1
  Modified Time: 1574911
  Debug: Off
  Object Name: 
  Observers: 
    none
ParameterMap 0: 
  (CenterOfRotationPoint -14.7098 -19.9938 56.6948)
  (CompressResultImage "false")
  (ComputeZYX "false")
  (DefaultPixelValue 0)
  (Direction 1 0 0 0 1 0 0 0 1)
  (FinalBSplineInterpolationOrder 3)
  (FixedImageDimension 3)
  (FixedInternalImagePixelType "float")
  (HowToCombineTransforms "Compose")
  (Index 0 0 0)
  (InitialTransformParametersFileName "NoInitialTransform")
  (MovingImageDimension 3)
  (MovingInternalImagePixelType "float")
  (NumberOfParameters 6)
  (Origin -194.013 -199.297 40.1948)
  (ResampleInterpolator "FinalBSplineInterpolator")
  (Resampler "DefaultResampler")
  (ResultImageFormat "nii")
  (ResultImagePixelType "float")
  (Size 256 256 4)
  (Spacing 1.4063 1.4063 11)
  (Transform "EulerTransform")
  (TransformParameters 0.000179208 0.

...

  (UseDirectionCosines "true")
Resizing t1_pre_in to (256, 256, 4)
Resizing t1_pre_water to (256, 256, 4)
Resizing t1_pre_out to (256, 256, 4)
Resizing t1_pre_fat to (256, 256, 4)
Resizing t2 to (256, 256, 4)
Resizing mask to (256, 256, 4)
Resizing mre_raw to (256, 256, 4)
Resizing wave to (256, 256, 4)
Resizing mre to (256, 256, 4)
Resizing mask to (256, 256, 4)
Correcting metadata on t1_pre_in
Correcting metadata on t1_pre_water
Correcting metadata on t1_pre_out
Correcting metadata on t1_pre_fat
Correcting metadata on t2
Correcting metadata on mask
Correcting metadata on mre_raw
Correcting metadata on wave
Correcting metadata on mre
Correcting metadata on mask
 97%|█████████▋| 151/155 [3:36:44<04:51, 73.00s/it]Correcting metadata on mre_raw
Correcting metadata on wave
Restoring wave
Registering t1_pre_out to mre_raw
ParameterObject (0x5591298a9730)
  RTTI typeinfo:   elastix::ParameterObject
  Reference Count: 1
  Modified Time: 41988011
  Debug: Off
  Object Name: 
  Observers: 
   

ParameterObject (0x5591309e6940)
  RTTI typeinfo:   elastix::ParameterObject
  Reference Count: 1
  Modified Time: 42227740
  Debug: Off
  Object Name: 
  Observers: 
    none
ParameterMap 0: 
  (CenterOfRotationPoint -11.2823 -20.0033 25.7564)
  (CompressResultImage "false")
  (ComputeZYX "false")
  (DefaultPixelValue 0)
  (Direction 1 0 0 0 1 0 0 0 1)
  (FinalBSplineInterpolationOrder 3)
  (FixedImageDimension 3)
  (FixedInternalImagePixelType "float")
  (HowToCombineTransforms "Compose")
  (Index 0 0 0)
  (InitialTransformParametersFileName "NoInitialTransform")
  (MovingImageDimension 3)
  (MovingInternalImagePixelType "float")
  (NumberOfParameters 6)
  (Origin -180.615 -189.336 -79.2436)
  (ResampleInterpolator "FinalBSplineInterpolator")
  (Resampler "DefaultResampler")
  (ResultImageFormat "nii")
  (ResultImagePixelType "float")
  (Size 256 256 22)
  (Spacing 1.3281 1.3281 10)
  (Transform "EulerTransform")
  (TransformParameters 0.0430698 0.0186638 0.00457146 0.0924042 0.73158

ParameterObject (0x559115566000)
  RTTI typeinfo:   elastix::ParameterObject
  Reference Count: 1
  Modified Time: 42467863
  Debug: Off
  Object Name: 
  Observers: 
    none
ParameterMap 0: 
  (CenterOfRotationPoint -3.83451 -37.1915 197.923)
  (CompressResultImage "false")
  (ComputeZYX "false")
  (DefaultPixelValue 0)
  (Direction 1 0 0 0 1 0 0 0 1)
  (FinalBSplineInterpolationOrder 3)
  (FixedImageDimension 3)
  (FixedInternalImagePixelType "float")
  (HowToCombineTransforms "Compose")
  (Index 0 0 0)
  (InitialTransformParametersFileName "NoInitialTransform")
  (MovingImageDimension 3)
  (MovingInternalImagePixelType "float")
  (NumberOfParameters 6)
  (Origin -213.421 -246.778 93.2729)
  (ResampleInterpolator "FinalBSplineInterpolator")
  (Resampler "DefaultResampler")
  (ResultImageFormat "nii")
  (ResultImagePixelType "float")
  (Size 512 512 92)
  (Spacing 0.8203 0.8203 2.3)
  (Transform "EulerTransform")
  (TransformParameters -0.0423964 -0.030814 0.0135387 2.71425 -6.10869 

ParameterObject (0x55912d41b340)
  RTTI typeinfo:   elastix::ParameterObject
  Reference Count: 1
  Modified Time: 42708109
  Debug: Off
  Object Name: 
  Observers: 
    none
ParameterMap 0: 
  (CenterOfRotationPoint -6.12225 -45.7823 161.542)
  (CompressResultImage "false")
  (ComputeZYX "false")
  (DefaultPixelValue 0)
  (Direction 1 0 0 0 1 0 0 0 1)
  (FinalBSplineInterpolationOrder 3)
  (FixedImageDimension 3)
  (FixedInternalImagePixelType "float")
  (HowToCombineTransforms "Compose")
  (Index 0 0 0)
  (InitialTransformParametersFileName "NoInitialTransform")
  (MovingImageDimension 3)
  (MovingInternalImagePixelType "float")
  (NumberOfParameters 6)
  (Origin -205.341 -245.001 145.042)
  (ResampleInterpolator "FinalBSplineInterpolator")
  (Resampler "DefaultResampler")
  (ResultImageFormat "nii")
  (ResultImagePixelType "float")
  (Size 256 256 4)
  (Spacing 1.5625 1.5625 11)
  (Transform "EulerTransform")
  (TransformParameters 0.000369008 0.000102775 -8.40163e-05 -0.00437215 -

ParameterObject (0x55911e18b240)
  RTTI typeinfo:   elastix::ParameterObject
  Reference Count: 1
  Modified Time: 42948367
  Debug: Off
  Object Name: 
  Observers: 
    none
ParameterMap 0: 
  (CenterOfRotationPoint 16.225 -19.997 -75.6528)
  (CompressResultImage "false")
  (ComputeZYX "false")
  (DefaultPixelValue 0)
  (Direction 1 0 0 0 1 0 0 0 1)
  (FinalBSplineInterpolationOrder 3)
  (FixedImageDimension 3)
  (FixedInternalImagePixelType "float")
  (HowToCombineTransforms "Compose")
  (Index 0 0 0)
  (InitialTransformParametersFileName "NoInitialTransform")
  (MovingImageDimension 3)
  (MovingInternalImagePixelType "float")
  (NumberOfParameters 6)
  (Origin -183.397 -219.619 -203.302)
  (ResampleInterpolator "FinalBSplineInterpolator")
  (Resampler "DefaultResampler")
  (ResultImageFormat "nii")
  (ResultImagePixelType "float")
  (Size 512 512 112)
  (Spacing 0.7813 0.7813 2.29999)
  (Transform "EulerTransform")
  (TransformParameters -0.0371107 -0.00337459 0.00594419 0.0416353 

In [28]:
%%time
dataset = mre_pinn.data.MREDataset.from_cohort(cohort)

Converting t1_pre_in to xarray
Converting t1_pre_water to xarray
Converting t1_pre_out to xarray
Converting t1_pre_fat to xarray
Converting t2 to xarray
Converting mre_raw to xarray
Converting wave to xarray
Converting mre to xarray
Converting mask to xarray
Converting mask to xarray
Converting t1_pre_in to xarray
Converting t1_pre_water to xarray
Converting t1_pre_out to xarray
Converting t1_pre_fat to xarray
Converting t2 to xarray
Converting mre_raw to xarray
Converting wave to xarray
Converting mre to xarray
Converting mask to xarray
Converting mask to xarray
Converting t1_pre_in to xarray
Converting t1_pre_water to xarray
Converting t1_pre_out to xarray
Converting t1_pre_fat to xarray
Converting t2 to xarray
Converting mre_raw to xarray
Converting wave to xarray
Converting mre to xarray
Converting mask to xarray
Converting mask to xarray
Converting t1_pre_in to xarray
Converting t1_pre_water to xarray
Converting t1_pre_out to xarray
Converting t1_pre_fat to xarray
Converting t2 to

Converting t1_pre_in to xarray
Converting t1_pre_water to xarray
Converting t1_pre_out to xarray
Converting t1_pre_fat to xarray
Converting t2 to xarray
Converting mre_raw to xarray
Converting wave to xarray
Converting mre to xarray
Converting mask to xarray
Converting mask to xarray
Converting t1_pre_in to xarray
Converting t1_pre_water to xarray
Converting t1_pre_out to xarray
Converting t1_pre_fat to xarray
Converting t2 to xarray
Converting mre_raw to xarray
Converting wave to xarray
Converting mre to xarray
Converting mask to xarray
Converting mask to xarray
Converting t1_pre_in to xarray
Converting t1_pre_water to xarray
Converting t1_pre_out to xarray
Converting t1_pre_fat to xarray
Converting t2 to xarray
Converting mre_raw to xarray
Converting wave to xarray
Converting mre to xarray
Converting mask to xarray
Converting mask to xarray
Converting t1_pre_in to xarray
Converting t1_pre_water to xarray
Converting t1_pre_out to xarray
Converting t1_pre_fat to xarray
Converting t2 to

Converting t1_pre_in to xarray
Converting t1_pre_water to xarray
Converting t1_pre_out to xarray
Converting t1_pre_fat to xarray
Converting t2 to xarray
Converting mre_raw to xarray
Converting wave to xarray
Converting mre to xarray
Converting mask to xarray
Converting mask to xarray
Converting t1_pre_in to xarray
Converting t1_pre_water to xarray
Converting t1_pre_out to xarray
Converting t1_pre_fat to xarray
Converting t2 to xarray
Converting mre_raw to xarray
Converting wave to xarray
Converting mre to xarray
Converting mask to xarray
Converting mask to xarray
Converting t1_pre_in to xarray
Converting t1_pre_water to xarray
Converting t1_pre_out to xarray
Converting t1_pre_fat to xarray
Converting t2 to xarray
Converting mre_raw to xarray
Converting wave to xarray
Converting mre to xarray
Converting mask to xarray
Converting mask to xarray
Converting t1_pre_in to xarray
Converting t1_pre_water to xarray
Converting t1_pre_out to xarray
Converting t1_pre_fat to xarray
Converting t2 to

Converting t1_pre_in to xarray
Converting t1_pre_water to xarray
Converting t1_pre_out to xarray
Converting t1_pre_fat to xarray
Converting t2 to xarray
Converting mre_raw to xarray
Converting wave to xarray
Converting mre to xarray
Converting mask to xarray
Converting mask to xarray
Converting t1_pre_in to xarray
Converting t1_pre_water to xarray
Converting t1_pre_out to xarray
Converting t1_pre_fat to xarray
Converting t2 to xarray
Converting mre_raw to xarray
Converting wave to xarray
Converting mre to xarray
Converting mask to xarray
Converting mask to xarray
Converting t1_pre_in to xarray
Converting t1_pre_water to xarray
Converting t1_pre_out to xarray
Converting t1_pre_fat to xarray
Converting t2 to xarray
Converting mre_raw to xarray
Converting wave to xarray
Converting mre to xarray
Converting mask to xarray
Converting mask to xarray
Converting t1_pre_in to xarray
Converting t1_pre_water to xarray
Converting t1_pre_out to xarray
Converting t1_pre_fat to xarray
Converting t2 to

CPU times: user 1.9 s, sys: 426 ms, total: 2.32 s
Wall time: 5 s


In [29]:
dataset.save_xarrays('../data/NAFLD/v4')

Writing ../data/NAFLD/v4/0898/wave.nc
Writing ../data/NAFLD/v4/0898/mre.nc
Writing ../data/NAFLD/v4/0898/mre_mask.nc
Writing ../data/NAFLD/v4/0898/anat.nc
Writing ../data/NAFLD/v4/0898/anat_mask.nc
Writing ../data/NAFLD/v4/0900/wave.nc
Writing ../data/NAFLD/v4/0900/mre.nc
Writing ../data/NAFLD/v4/0900/mre_mask.nc
Writing ../data/NAFLD/v4/0900/anat.nc
Writing ../data/NAFLD/v4/0900/anat_mask.nc
Writing ../data/NAFLD/v4/0929/wave.nc
Writing ../data/NAFLD/v4/0929/mre.nc
Writing ../data/NAFLD/v4/0929/mre_mask.nc
Writing ../data/NAFLD/v4/0929/anat.nc
Writing ../data/NAFLD/v4/0929/anat_mask.nc
Writing ../data/NAFLD/v4/0931/wave.nc
Writing ../data/NAFLD/v4/0931/mre.nc
Writing ../data/NAFLD/v4/0931/mre_mask.nc
Writing ../data/NAFLD/v4/0931/anat.nc
Writing ../data/NAFLD/v4/0931/anat_mask.nc
Writing ../data/NAFLD/v4/0932/wave.nc
Writing ../data/NAFLD/v4/0932/mre.nc
Writing ../data/NAFLD/v4/0932/mre_mask.nc
Writing ../data/NAFLD/v4/0932/anat.nc
Writing ../data/NAFLD/v4/0932/anat_mask.nc
Writing ..

Writing ../data/NAFLD/v4/1337/anat_mask.nc
Writing ../data/NAFLD/v4/1341/wave.nc
Writing ../data/NAFLD/v4/1341/mre.nc
Writing ../data/NAFLD/v4/1341/mre_mask.nc
Writing ../data/NAFLD/v4/1341/anat.nc
Writing ../data/NAFLD/v4/1341/anat_mask.nc
Writing ../data/NAFLD/v4/1343/wave.nc
Writing ../data/NAFLD/v4/1343/mre.nc
Writing ../data/NAFLD/v4/1343/mre_mask.nc
Writing ../data/NAFLD/v4/1343/anat.nc
Writing ../data/NAFLD/v4/1343/anat_mask.nc
Writing ../data/NAFLD/v4/1344/wave.nc
Writing ../data/NAFLD/v4/1344/mre.nc
Writing ../data/NAFLD/v4/1344/mre_mask.nc
Writing ../data/NAFLD/v4/1344/anat.nc
Writing ../data/NAFLD/v4/1344/anat_mask.nc
Writing ../data/NAFLD/v4/1382/wave.nc
Writing ../data/NAFLD/v4/1382/mre.nc
Writing ../data/NAFLD/v4/1382/mre_mask.nc
Writing ../data/NAFLD/v4/1382/anat.nc
Writing ../data/NAFLD/v4/1382/anat_mask.nc
Writing ../data/NAFLD/v4/1395/wave.nc
Writing ../data/NAFLD/v4/1395/mre.nc
Writing ../data/NAFLD/v4/1395/mre_mask.nc
Writing ../data/NAFLD/v4/1395/anat.nc
Writing ..

Writing ../data/NAFLD/v4/0234/mre.nc
Writing ../data/NAFLD/v4/0234/mre_mask.nc
Writing ../data/NAFLD/v4/0234/anat.nc
Writing ../data/NAFLD/v4/0234/anat_mask.nc
Writing ../data/NAFLD/v4/0235/wave.nc
Writing ../data/NAFLD/v4/0235/mre.nc
Writing ../data/NAFLD/v4/0235/mre_mask.nc
Writing ../data/NAFLD/v4/0235/anat.nc
Writing ../data/NAFLD/v4/0235/anat_mask.nc
Writing ../data/NAFLD/v4/0457/wave.nc
Writing ../data/NAFLD/v4/0457/mre.nc
Writing ../data/NAFLD/v4/0457/mre_mask.nc
Writing ../data/NAFLD/v4/0457/anat.nc
Writing ../data/NAFLD/v4/0457/anat_mask.nc
Writing ../data/NAFLD/v4/0461/wave.nc
Writing ../data/NAFLD/v4/0461/mre.nc
Writing ../data/NAFLD/v4/0461/mre_mask.nc
Writing ../data/NAFLD/v4/0461/anat.nc
Writing ../data/NAFLD/v4/0461/anat_mask.nc
Writing ../data/NAFLD/v4/0491/wave.nc
Writing ../data/NAFLD/v4/0491/mre.nc
Writing ../data/NAFLD/v4/0491/mre_mask.nc
Writing ../data/NAFLD/v4/0491/anat.nc
Writing ../data/NAFLD/v4/0491/anat_mask.nc
Writing ../data/NAFLD/v4/0492/wave.nc
Writing ..

Writing ../data/NAFLD/v4/1748/anat_mask.nc
Writing ../data/NAFLD/v4/1785/wave.nc
Writing ../data/NAFLD/v4/1785/mre.nc
Writing ../data/NAFLD/v4/1785/mre_mask.nc
Writing ../data/NAFLD/v4/1785/anat.nc
Writing ../data/NAFLD/v4/1785/anat_mask.nc
Writing ../data/NAFLD/v4/1786/wave.nc
Writing ../data/NAFLD/v4/1786/mre.nc
Writing ../data/NAFLD/v4/1786/mre_mask.nc
Writing ../data/NAFLD/v4/1786/anat.nc
Writing ../data/NAFLD/v4/1786/anat_mask.nc
Writing ../data/NAFLD/v4/1790/wave.nc
Writing ../data/NAFLD/v4/1790/mre.nc
Writing ../data/NAFLD/v4/1790/mre_mask.nc
Writing ../data/NAFLD/v4/1790/anat.nc
Writing ../data/NAFLD/v4/1790/anat_mask.nc
Writing ../data/NAFLD/v4/1791/wave.nc
Writing ../data/NAFLD/v4/1791/mre.nc
Writing ../data/NAFLD/v4/1791/mre_mask.nc
Writing ../data/NAFLD/v4/1791/anat.nc
Writing ../data/NAFLD/v4/1791/anat_mask.nc
Writing ../data/NAFLD/v4/1793/wave.nc
Writing ../data/NAFLD/v4/1793/mre.nc
Writing ../data/NAFLD/v4/1793/mre_mask.nc
Writing ../data/NAFLD/v4/1793/anat.nc
Writing ..

In [ ]:
%autoreload
dataset[0].view('wave', 'mre', 'anat', mask=True)